<h2>Moonboard Problem Grade Calculator</h2>
<p>Bouldering is a physically demanding sport, that relies on the categorization of climbing routes into various difficulty levels. However, the process of grading rock climbing routes is inherently subjective and can vary significantly based on factors such as the climber's experience, the specific climbing holds, and routes. This subjectivity poses challenges for route setters aiming to accurately convey the nature of the climb to the climbing community.</p>
<p>The Moonboard is a standardzied training device which removes some variability of the factors invovled in the sport. The Moonboard can provide a standardized framework to evaluate the multifaceted aspects that contribute to a route's difficulty. Factors such as hold types, overhang angles, and route length can be quantified and integrated into a comprehensive equation.</p>
<p></p>

In [39]:
import tensorflow as tf
from converter import *

Step 1: Convert the grades of the problems from Fontainebleau scale to TF scale. Create the grade list to be used in the model.

In [40]:
raw_grades = open('../Raw Data/grades.txt', 'r')
converted_grades = open('../TF Data/TF grades.txt', 'w')
grade_list = []
for problem in raw_grades:
    problem = problem.strip()
    grade = Grade(problem)
    grade.grade_converter(grade, grade_list, converted_grades)
converted_grades.close()
grade_array = np.array(grade_list)

Step 2: Create array with 3 layers for each hold position to be used as the nueral network training data:
<li> Legal hold positions</li>
<li> Start position(s)</li>
<li> Finish position(s)</li>


In [41]:
h = open('../Raw Data/holds.txt')
sh = open('../Raw Data/start holds.txt')
eh = open('../Raw Data/end holds.txt')
out_file = open('../TF Data/Arrays.txt', 'w')
problem_array = create_array(h,sh, eh)
close_files(h, sh, eh, out_file)

Step 3: Build the nueral network with the training set.

In [42]:
# Set random seed
tf.random.set_seed(42)

# Create the model
model= tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(problem_array.shape[1],problem_array.shape[2],problem_array.shape[3])),
    tf.keras.layers.Dense(2048, activation="relu"),
    tf.keras.layers.Dense(1024, activation="relu"),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(16, activation="softmax")
])

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00005)
model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Fit the model
model_fit = model.fit(problem_array, grade_array, epochs = 40)

Epoch 1/40
27/27 [==============================] - 3s 60ms/step - loss: 2.7204 - accuracy: 0.1721
Epoch 2/40
27/27 [==============================] - 2s 59ms/step - loss: 2.5574 - accuracy: 0.2272
Epoch 3/40
27/27 [==============================] - 2s 64ms/step - loss: 2.3483 - accuracy: 0.2131
Epoch 4/40
27/27 [==============================] - 2s 59ms/step - loss: 2.1417 - accuracy: 0.2635
Epoch 5/40
27/27 [==============================] - 2s 59ms/step - loss: 1.9029 - accuracy: 0.4040
Epoch 6/40
27/27 [==============================] - 2s 61ms/step - loss: 1.6355 - accuracy: 0.5773
Epoch 7/40
27/27 [==============================] - 2s 61ms/step - loss: 1.3607 - accuracy: 0.7459
Epoch 8/40
27/27 [==============================] - 2s 62ms/step - loss: 1.1191 - accuracy: 0.8056
Epoch 9/40
27/27 [==============================] - 2s 60ms/step - loss: 0.8945 - accuracy: 0.8677
Epoch 10/40
27/27 [==============================] - 2s 57ms/step - loss: 0.7196 - accuracy: 0.9180
Epoch 11/

To test a problem, edit the file Test_Problem.txt in the TF folder.
<li> Change the starting hold to "S"</li>
<li> Change the finish hold to "F"</li>
<li> Change any other legal holds to "1"</li>

In [46]:
test_doc = open('../TF Data/Test_Problem.txt')
# Build the input array
rows = [0] * 11
matrix = [rows] * 18
holds = np.array([rows] * 18)
start = np.array([rows] * 18)
end = np.array([rows] * 18)
k = 0
# Iterate through and adjust empty matricies for holds
for row in test_doc.readlines():
    row = row.replace(" ", "")
    j = 0
    for hold in row:
        if hold == '1':
            holds[k][j] = 1
            start[k][j] = 0
            end[k][j] = 0
        if hold == 'S':
            holds[k][j] = 1
            start[k][j] = 1
            end[k][j] = 0
        if hold == 'F':
            holds[k][j] = 1
            start[k][j] = 0
            end[k][j] = 1
        if hold == '\n':
            pass
        j = j + 1
    k = k + 1
test_array = []
test_array.append(np.stack((holds,start,end)))
test_array = np.array(test_array)
# Predict the grade
y_probs = model.predict(test_array)
d = tf.argmax(y_probs[0])
r = Grade(d)
l = r.score_to_font()
print(l)

1/1 [==============================] - 0s 22ms/step
7C
